# 01_data_parsing.ipynb

В этом ноутбуке собираются данные о новостройках Москвы.

Данные получены с помощью парсинга с сайта Move.ru.

Результат ноутбука — таблица с сырыми объектами (цены, площади, адреса, девелопер и др.),
которая используется в дальнейшей обработке и анализе.


# Парсинг данных с сайта Move.ru

https://move.ru/kvartiry_v_novostroykah/v_predelah_mkad/



In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import time
import random
import logging
from fake_useragent import UserAgent
ua = UserAgent()

def get_page_with_retry(url, max_retries=3):
    for attempt in range(max_retries):
        try:
            headers = {
                'User-Agent': ua.random,
                'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
                'Accept-Language': 'ru-RU,ru;q=0.8,en-US;q=0.5,en;q=0.3',
                'Accept-Encoding': 'gzip, deflate, br',
                'Connection': 'keep-alive',
                'Upgrade-Insecure-Requests': '1',
            }

            response = requests.get(
                url,
                headers=headers,
                timeout=10,
                allow_redirects=True
            )

            if response.status_code == 200:
                return response
            else:
                print(f"Попытка {attempt + 1}: Статус код {response.status_code} для {url}")

        except requests.exceptions.RequestException as e:
            print(f"Попытка {attempt + 1}: Ошибка при запросе {url}: {e}")

        if attempt < max_retries - 1:
            time.sleep(random.uniform(2, 5))

    return None

def extract_urls_from_page(soup, base_url):
    urls = []
    for link in soup.find_all('a', href=True):
        href = link['href']
        if re.search(r'/objects/[\w-]+\s*[\w\s]*\d+', href):
            absolute_url = urljoin(base_url, href)
            if absolute_url not in urls:
                urls.append(absolute_url)
    return urls

def main():
    base_url = 'https://move.ru/kvartiry_v_novostroykah/v_predelah_mkad/'
    all_urls = []
    max_pages = 400

    for page_num in range(1, max_pages + 1):
        print(f"Обрабатывается страница {page_num} из {max_pages}...")

        if page_num == 1:
            page_url = base_url
        else:
            page_url = f"{base_url}?page={page_num}"

        # Получаем страницу
        response = get_page_with_retry(page_url)

        if response is None:
            print(f"Не удалось получить страницу {page_num}, пропускаем...")
            continue

        # Парсим HTML
        soup = BeautifulSoup(response.text, 'html.parser')

        # Извлекаем URL
        page_urls = extract_urls_from_page(soup, base_url)
        all_urls.extend(page_urls)

        # Случайная задержка между запросами
        time.sleep(random.uniform(1, 3))

    # Удаляем дубликаты
    all_urls = list(set(all_urls))

    # Сохраняем в файл
    with open('extracted_urls.txt', 'w', encoding='utf-8') as f:
        for url in all_urls:
            f.write(url + '\n')

    print(f"Результаты сохранены в файл 'extracted_urls.txt'")

if name == "main":
    main()